## Análisis de accidentes - Buses

### Area metropolitana del Valle de Aburrá

Manuela Cordoba

Alejandro Fernandez

Ana María Jaramillo


In [1]:
import pandas as pd
import re

pd.set_option("display.max_columns", None)


import config

In [2]:
# Define a function to clean the values
def clean_col(value: str) -> str:
    """
    This function cleans the values of a column.
    Args:
        value: value with accents, special characters, etc.
    Returns:
        value: value without accents, special characters, etc.
    """
    if isinstance(value, str):
        value = value.lower()
        value = re.sub(r"[^\w\s]", "", value)
        value = value.replace("ñ", "n")
        value = value.replace("á", "a")
        value = value.replace("é", "e")
        value = value.replace("í", "i")
        value = value.replace("ó", "o")
        value = value.replace("ú", "u")
        value = re.sub(r"\s+", " ", value)
        value = re.sub(r"\s+$", "", value)
        value = value.replace(" ", "_")
    return value


# Function to replace values
def replace_malla(value: str) -> str:
    """
    This function replaces the values of a column. If the value starts with "malla_vial_aproximada_" it will be replaced by "malla_vial_aproximada".
    Args:
        value: value to replace.
    Returns:
        value: value replaced.
    """
    if value.startswith("malla_vial_aproximada_"):
        return "malla_vial_aproximada"
    return value

In [3]:
vehiculos_2013_2023 = pd.read_excel(config.BASE_DATOS_INICIAL)
print("Shape vehiculos_2013_2023: ", vehiculos_2013_2023.shape)
vehiculos_2013_2023.head()

Shape vehiculos_2013_2023:  (741858, 16)


,NRO_RADICADO,AÑO_ACCIDENTE,CLASE_ACCIDENTE,GRAVEDAD_ACCIDENTE,CLASE_VEHICULO,TIPO_SERVICIO,MARCA,MODELO,ZONA,Diseño,EDAD_CONDUCTOR,SEXO,Mes,Día,Num Día,Hora
0,1474935,2015,Choque,SOLO DAÑOS,AUTOMOVIL,PARTICULAR,FORD,2011.0,ZONA DESCONOCIDA,Tramo de via,45,Masculino,Feb,Fri,6,7
1,1474935,2015,Choque,SOLO DAÑOS,MOTOCICLETA,PARTICULAR,AKT,2000.0,ZONA DESCONOCIDA,Tramo de via,45,Sin inf,Feb,Vie,6,7
2,1503730,2015,Otro,HERIDO,MOTOCICLETA,PARTICULAR,YAMAHA,2010.0,Jurisdicción,Tramo de via,24,Masculino,Sep,Jue,10,15
3,1503732,2015,Atropello,HERIDO,MOTOCICLETA,DESCONOCIDO,NaN,NaN,Jurisdicción,Tramo de via,35,Masculino,Sep,Dom,13,15
4,1503734,2015,Atropello,HERIDO,MOTOCICLETA,PARTICULAR,AUTECO,2010.0,Jurisdicción,Tramo de via,48,Masculino,Sep,Lun,14,15


##### NOTA DE JUAN PINEDA: Aqui arriba en vehiculos_2013_2023 veo que es la tabla inicial con todos los modos, entonces no entiendo para que filtraron en el archivo 2 solo para los buses. Verifiquen por favor.
##### RESPUESTA: El archivo 2 nos permite obtener los radicados involucrados en accidentes con buses, pero el registro aparece duplicado con los demas vehiculos implicados, que pueden ser automovil, motocicleta, etc.

In [4]:
more_attributes = pd.read_excel(config.ATRIBUTOS_GEOREFERENCIADOS)
print("Shape more_attributes: ", more_attributes.shape)
more_attributes.head()

Shape more_attributes:  (45563, 20)


,RADICADO,HORA,PERIODO,CLASE,TIPO_GEOCO,GRAVEDAD,BARRIO,COMUNA,DISENO,MES,DIA,LONGITUD,LATITUD,areagralus,cod_cat_us,cod_subcat,subcategor,HIERARCHY,ROAD_CLASS,ESTRATO_PR
0,1739400,2:20:00 p. m.,2021,Caida Ocupante,Malla vial,CON HERIDOS,Prado,La Candelaria,Glorieta,2,25,-75.563376,6.263505,?reas y corredores de alta mixtura,3.0,33.0,Centralidades con predominancia econ?mica,3.0,6.0,SIN_ESTRATO
1,1735512,6:20:00 p. m.,2021,Choque,Malla vial,SOLO DAÑOS,El Progreso,Castilla,Tramo de via,1,14,-75.578933,6.271674,?reas y corredores de media mixtura,2.0,21.0,Zonas de Transici?n; Centralidades dotacionale...,4.0,1.0,TRES
2,1735374,7:30:00 p. m.,2021,Choque,Malla vial,SOLO DAÑOS,Caribe,Castilla,Tramo de via,1,13,-75.575081,6.266311,?reas y corredores de alta mixtura,3.0,31.0,?reas de actividad econ?mica en trasformaci?n,3.0,6.0,SIN_ESTRATO
3,1735813,6:40:00 p. m.,2021,Choque,EPM sin Interior,SOLO DAÑOS,La Candelaria,La Candelaria,Interseccion,1,19,-75.569255,6.247219,?reas y corredores de alta mixtura,3.0,33.0,Centralidades con predominancia econ?mica,5.0,1.0,SIN_ESTRATO
4,1736179,3:30:00 p. m.,2021,Choque,Malla vial,SOLO DAÑOS,Prado,La Candelaria,Tramo de via,1,23,-75.559221,6.260778,?reas y corredores de alta mixtura,3.0,32.0,Centralidades y corredores con alta mixtura,4.0,1.0,CUATRO


In [5]:
joined_data = more_attributes.merge(
    vehiculos_2013_2023, left_on="RADICADO", right_on="NRO_RADICADO"
)
print("Shape of joined_data: ", joined_data.shape)
joined_data.tail()

Shape of joined_data:  (87649, 36)


,RADICADO,HORA,PERIODO,CLASE,TIPO_GEOCO,GRAVEDAD,BARRIO,COMUNA,DISENO,MES,DIA,LONGITUD,LATITUD,areagralus,cod_cat_us,cod_subcat,subcategor,HIERARCHY,ROAD_CLASS,ESTRATO_PR,NRO_RADICADO,AÑO_ACCIDENTE,CLASE_ACCIDENTE,GRAVEDAD_ACCIDENTE,CLASE_VEHICULO,TIPO_SERVICIO,MARCA,MODELO,ZONA,Diseño,EDAD_CONDUCTOR,SEXO,Mes,Día,Num Día,Hora
87644,1470979,04:50 PM,2014,Choque,Malla vial aproximada: CL 47-50,SOLO DAÑOS,La Candelaria,La Candelaria,Tramo de via,12,31,-75.568715,6.247466,?reas y corredores de alta mixtura,3.0,33.0,Centralidades con predominancia econ?mica,5.0,1.0,SIN_ESTRATO,1470979,2014,Choque,SOLO DAÑOS,MICRO BUS,PUBLICO,DAIHATSU,2006.0,COMUNA 10,Tramo de via,49,Masculino,Dic,Mié,31,16
87645,1470984,01:10 PM,2014,Choque,Malla vial cruce invertido,SOLO DAÑOS,La Candelaria,La Candelaria,Tramo de via,12,31,-75.564683,6.249282,?reas y corredores de alta mixtura,3.0,33.0,Centralidades con predominancia econ?mica,3.0,6.0,CUATRO,1470984,2014,Choque,SOLO DAÑOS,AUTOMOVIL,PUBLICO,HYUNDAI,2014.0,COMUNA 10,Tramo de via,22,Masculino,Dic,Mié,31,13
87646,1470984,01:10 PM,2014,Choque,Malla vial cruce invertido,SOLO DAÑOS,La Candelaria,La Candelaria,Tramo de via,12,31,-75.564683,6.249282,?reas y corredores de alta mixtura,3.0,33.0,Centralidades con predominancia econ?mica,3.0,6.0,CUATRO,1470984,2014,Choque,SOLO DAÑOS,BUS,PUBLICO,MODASA,2013.0,COMUNA 10,Tramo de via,41,Masculino,Dic,Mié,31,13
87647,1470985,03:20 PM,2014,Choque,Malla vial aproximada: CL 57-19,SOLO DAÑOS,Los Mangos,Villa Hermosa,Ciclo Ruta,12,31,-75.541845,6.249259,?reas de baja mixtura,1.0,11.0,?reas predominantemente residenciales,5.0,1.0,UNO,1470985,2014,Choque,SOLO DAÑOS,AUTOMOVIL,PARTICULAR,CHEVROLET,2009.0,COMUNA 8,Ciclo Ruta,41,Masculino,Dic,Mié,31,15
87648,1470985,03:20 PM,2014,Choque,Malla vial aproximada: CL 57-19,SOLO DAÑOS,Los Mangos,Villa Hermosa,Ciclo Ruta,12,31,-75.541845,6.249259,?reas de baja mixtura,1.0,11.0,?reas predominantemente residenciales,5.0,1.0,UNO,1470985,2014,Choque,SOLO DAÑOS,MICRO BUS,PUBLICO,CHEVROLET,2014.0,COMUNA 8,Ciclo Ruta,25,Masculino,Dic,Mié,31,15


In [6]:
print(joined_data.columns)

Index(['RADICADO', 'HORA', 'PERIODO', 'CLASE', 'TIPO_GEOCO', 'GRAVEDAD',
       'BARRIO', 'COMUNA', 'DISENO', 'MES', 'DIA', 'LONGITUD', 'LATITUD',
       'areagralus', 'cod_cat_us', 'cod_subcat', 'subcategor', 'HIERARCHY',
       'ROAD_CLASS', 'ESTRATO_PR', 'NRO_RADICADO', 'AÑO_ACCIDENTE',
       'CLASE_ACCIDENTE', 'GRAVEDAD_ACCIDENTE', 'CLASE_VEHICULO',
       'TIPO_SERVICIO', 'MARCA', 'MODELO', 'ZONA', 'Diseño', 'EDAD_CONDUCTOR',
       'SEXO', 'Mes', 'Día', 'Num Día', 'Hora'],
      dtype='object')


### Drop duplicated columns


In [7]:
joined_data.drop(
    columns=[
        "HORA",
        "COMUNA",
        "CLASE",
        "DISENO",
        "NRO_RADICADO",
        "AÑO_ACCIDENTE",
        "GRAVEDAD_ACCIDENTE",
        "Mes",
        "Num Día",
    ],
    inplace=True,
)
joined_data.head()

,RADICADO,PERIODO,TIPO_GEOCO,GRAVEDAD,BARRIO,MES,DIA,LONGITUD,LATITUD,areagralus,cod_cat_us,cod_subcat,subcategor,HIERARCHY,ROAD_CLASS,ESTRATO_PR,CLASE_ACCIDENTE,CLASE_VEHICULO,TIPO_SERVICIO,MARCA,MODELO,ZONA,Diseño,EDAD_CONDUCTOR,SEXO,Día,Hora
0,1739400,2021,Malla vial,CON HERIDOS,Prado,2,25,-75.563376,6.263505,?reas y corredores de alta mixtura,3.0,33.0,Centralidades con predominancia econ?mica,3.0,6.0,SIN_ESTRATO,Caida Ocupante,BUS,PUBLICO,MODASA,2011.0,COMUNA 3,Glorieta,45,Masculino,Jue,14
1,1735512,2021,Malla vial,SOLO DAÑOS,El Progreso,1,14,-75.578933,6.271674,?reas y corredores de media mixtura,2.0,21.0,Zonas de Transici?n; Centralidades dotacionale...,4.0,1.0,TRES,Choque,AUTOMOVIL,PARTICULAR,MAZDA,2011.0,COMUNA 7,Tramo de via,30,Masculino,Jue,18
2,1735512,2021,Malla vial,SOLO DAÑOS,El Progreso,1,14,-75.578933,6.271674,?reas y corredores de media mixtura,2.0,21.0,Zonas de Transici?n; Centralidades dotacionale...,4.0,1.0,TRES,Choque,BUS,PUBLICO,AGRALE,2008.0,COMUNA 7,Tramo de via,58,Masculino,Jue,18
3,1735374,2021,Malla vial,SOLO DAÑOS,Caribe,1,13,-75.575081,6.266311,?reas y corredores de alta mixtura,3.0,31.0,?reas de actividad econ?mica en trasformaci?n,3.0,6.0,SIN_ESTRATO,Choque,AUTOMOVIL,PARTICULAR,RENAULT,2017.0,COMUNA 7,Tramo de via,55,Masculino,Mié,19
4,1735374,2021,Malla vial,SOLO DAÑOS,Caribe,1,13,-75.575081,6.266311,?reas y corredores de alta mixtura,3.0,31.0,?reas de actividad econ?mica en trasformaci?n,3.0,6.0,SIN_ESTRATO,Choque,BUS,PUBLICO,CHEVROLET,2015.0,COMUNA 7,Tramo de via,30,Masculino,Mié,19


In [8]:
joined_data.rename(columns={"Día": "dia_semana"}, inplace=True)

### Categories adjustment: Clase_vehiculo


In [9]:
## definicion de "otros" tipos de vehículo

joined_data["CLASE_VEHICULO"].replace(config.VALUES_TO_REPLACE, "OTROS", inplace=True)
joined_data["CLASE_VEHICULO"].unique()

array(['BUS', 'AUTOMOVIL', 'MOTOCICLETA', 'BUS ARTICULADO', 'BUSETA',
       'MICRO BUS', 'OTROS', 'CAMION', 'CAMIONETA', 'CAMPERO',
       'BICICLETA', 'MINI BUS', nan], dtype=object)

In [10]:
replacement_dict_clase_vehiculo = {
    "BUS ARTICULADO": "BUS",
    "BUSETA": "BUS",
    "MICRO BUS": "BUS",
    "MINI BUS": "BUS",
    "CAMIONETA": "AUTOMOVIL", 
    "CAMPERO": "AUTOMOVIL"
}

joined_data["CLASE_VEHICULO"].replace(replacement_dict_clase_vehiculo, inplace=True)

joined_data["CLASE_VEHICULO"].unique()

array(['BUS', 'AUTOMOVIL', 'MOTOCICLETA', 'OTROS', 'CAMION', 'BICICLETA',
       nan], dtype=object)

In [11]:
print("Joined data shape: ", joined_data.shape)

Joined data shape:  (87649, 27)


In [12]:
joined_data.columns

Index(['RADICADO', 'PERIODO', 'TIPO_GEOCO', 'GRAVEDAD', 'BARRIO', 'MES', 'DIA',
       'LONGITUD', 'LATITUD', 'areagralus', 'cod_cat_us', 'cod_subcat',
       'subcategor', 'HIERARCHY', 'ROAD_CLASS', 'ESTRATO_PR',
       'CLASE_ACCIDENTE', 'CLASE_VEHICULO', 'TIPO_SERVICIO', 'MARCA', 'MODELO',
       'ZONA', 'Diseño', 'EDAD_CONDUCTOR', 'SEXO', 'dia_semana', 'Hora'],
      dtype='object')

### Special characteres adjustment


In [13]:
# Create a mapping dictionary with cleaned column names
cleaned_columns = {col: clean_col(col) for col in joined_data.columns}

# Rename the columns using the mapping dictionary
joined_data.rename(columns=cleaned_columns, inplace=True)

joined_data.head()

,radicado,periodo,tipo_geoco,gravedad,barrio,mes,dia,longitud,latitud,areagralus,cod_cat_us,cod_subcat,subcategor,hierarchy,road_class,estrato_pr,clase_accidente,clase_vehiculo,tipo_servicio,marca,modelo,zona,diseno,edad_conductor,sexo,dia_semana,hora
0,1739400,2021,Malla vial,CON HERIDOS,Prado,2,25,-75.563376,6.263505,?reas y corredores de alta mixtura,3.0,33.0,Centralidades con predominancia econ?mica,3.0,6.0,SIN_ESTRATO,Caida Ocupante,BUS,PUBLICO,MODASA,2011.0,COMUNA 3,Glorieta,45,Masculino,Jue,14
1,1735512,2021,Malla vial,SOLO DAÑOS,El Progreso,1,14,-75.578933,6.271674,?reas y corredores de media mixtura,2.0,21.0,Zonas de Transici?n; Centralidades dotacionale...,4.0,1.0,TRES,Choque,AUTOMOVIL,PARTICULAR,MAZDA,2011.0,COMUNA 7,Tramo de via,30,Masculino,Jue,18
2,1735512,2021,Malla vial,SOLO DAÑOS,El Progreso,1,14,-75.578933,6.271674,?reas y corredores de media mixtura,2.0,21.0,Zonas de Transici?n; Centralidades dotacionale...,4.0,1.0,TRES,Choque,BUS,PUBLICO,AGRALE,2008.0,COMUNA 7,Tramo de via,58,Masculino,Jue,18
3,1735374,2021,Malla vial,SOLO DAÑOS,Caribe,1,13,-75.575081,6.266311,?reas y corredores de alta mixtura,3.0,31.0,?reas de actividad econ?mica en trasformaci?n,3.0,6.0,SIN_ESTRATO,Choque,AUTOMOVIL,PARTICULAR,RENAULT,2017.0,COMUNA 7,Tramo de via,55,Masculino,Mié,19
4,1735374,2021,Malla vial,SOLO DAÑOS,Caribe,1,13,-75.575081,6.266311,?reas y corredores de alta mixtura,3.0,31.0,?reas de actividad econ?mica en trasformaci?n,3.0,6.0,SIN_ESTRATO,Choque,BUS,PUBLICO,CHEVROLET,2015.0,COMUNA 7,Tramo de via,30,Masculino,Mié,19


In [14]:
joined_data.columns

Index(['radicado', 'periodo', 'tipo_geoco', 'gravedad', 'barrio', 'mes', 'dia',
       'longitud', 'latitud', 'areagralus', 'cod_cat_us', 'cod_subcat',
       'subcategor', 'hierarchy', 'road_class', 'estrato_pr',
       'clase_accidente', 'clase_vehiculo', 'tipo_servicio', 'marca', 'modelo',
       'zona', 'diseno', 'edad_conductor', 'sexo', 'dia_semana', 'hora'],
      dtype='object')

In [15]:
#  Checking string columns in joined_data
string_cols = joined_data.select_dtypes(include=["object"]).columns
string_cols

Index(['radicado', 'tipo_geoco', 'gravedad', 'barrio', 'areagralus',
       'subcategor', 'estrato_pr', 'clase_accidente', 'clase_vehiculo',
       'tipo_servicio', 'marca', 'zona', 'diseno', 'edad_conductor', 'sexo',
       'dia_semana'],
      dtype='object')

In [16]:
#  For all string_cols, apply astype(str) and then apply clean_col
for i in string_cols:
    joined_data[i] = joined_data[i].astype(str).apply(clean_col)

joined_data.head()

,radicado,periodo,tipo_geoco,gravedad,barrio,mes,dia,longitud,latitud,areagralus,cod_cat_us,cod_subcat,subcategor,hierarchy,road_class,estrato_pr,clase_accidente,clase_vehiculo,tipo_servicio,marca,modelo,zona,diseno,edad_conductor,sexo,dia_semana,hora
0,1739400,2021,malla_vial,con_heridos,prado,2,25,-75.563376,6.263505,reas_y_corredores_de_alta_mixtura,3.0,33.0,centralidades_con_predominancia_econmica,3.0,6.0,sin_estrato,caida_ocupante,bus,publico,modasa,2011.0,comuna_3,glorieta,45,masculino,jue,14
1,1735512,2021,malla_vial,solo_danos,el_progreso,1,14,-75.578933,6.271674,reas_y_corredores_de_media_mixtura,2.0,21.0,zonas_de_transicin_centralidades_dotacionales_...,4.0,1.0,tres,choque,automovil,particular,mazda,2011.0,comuna_7,tramo_de_via,30,masculino,jue,18
2,1735512,2021,malla_vial,solo_danos,el_progreso,1,14,-75.578933,6.271674,reas_y_corredores_de_media_mixtura,2.0,21.0,zonas_de_transicin_centralidades_dotacionales_...,4.0,1.0,tres,choque,bus,publico,agrale,2008.0,comuna_7,tramo_de_via,58,masculino,jue,18
3,1735374,2021,malla_vial,solo_danos,caribe,1,13,-75.575081,6.266311,reas_y_corredores_de_alta_mixtura,3.0,31.0,reas_de_actividad_econmica_en_trasformacin,3.0,6.0,sin_estrato,choque,automovil,particular,renault,2017.0,comuna_7,tramo_de_via,55,masculino,mie,19
4,1735374,2021,malla_vial,solo_danos,caribe,1,13,-75.575081,6.266311,reas_y_corredores_de_alta_mixtura,3.0,31.0,reas_de_actividad_econmica_en_trasformacin,3.0,6.0,sin_estrato,choque,bus,publico,chevrolet,2015.0,comuna_7,tramo_de_via,30,masculino,mie,19


### Categories adjustment: Sub_categor


In [17]:
replacement_dict_subcategor = {
    "centralidades_con_predominancia_econmica": "centralidades_con_predominancia_economica",
    "zonas_de_transicin_centralidades_dotacionales_aglomeraciones_comerciales_y_de_servicios_barriales": "zonas_de_transicion_centralidades_dotacionales_aglomeraciones_comerciales_y_de_servicios_barriales",
    "reas_de_actividad_econmica_en_trasformacin": "areas_de_actividad_economica_en_trasformacion",
    "centralidades_y_corredores_con_alta_mixtura": "centralidades_y_corredores_con_alta_mixtura",
    "reas_predominantemente_residenciales": "areas_predominantemente_residenciales",
    "zonas_verdes_publicas_sistema_de_movilidad": "zonas_verdes_publicas_sistema_de_movilidad",
    "zonas_verdes_publicas_sistema_hidrogrfico": "zonas_verdes_publicas_sistema_hidrografico",
}

joined_data["subcategor"].replace(replacement_dict_subcategor, inplace=True)

joined_data["subcategor"].unique()

array(['centralidades_con_predominancia_economica',
       'zonas_de_transicion_centralidades_dotacionales_aglomeraciones_comerciales_y_de_servicios_barriales',
       'areas_de_actividad_economica_en_trasformacion',
       'centralidades_y_corredores_con_alta_mixtura',
       'areas_predominantemente_residenciales', 'nan',
       'zonas_verdes_publicas_sistema_de_movilidad',
       'zonas_verdes_publicas_sistema_hidrografico'], dtype=object)

### Categories adjustment: Area_gral_us


In [18]:
replacement_dict_areagralus = {
    "reas_y_corredores_de_alta_mixtura": "areas_y_corredores_de_alta_mixtura",
    "reas_de_baja_mixtura": "areas_de_baja_mixtura",
    "reas_y_corredores_de_media_mixtura": "areas_y_corredores_de_media_mixtura",
    "espacio_pblico_existente": "espacio_publico_existente",
    "espacio_pblico_proyectado": "espacio_publico_proyectado",
}

joined_data["areagralus"].replace(replacement_dict_areagralus, inplace=True)


joined_data["areagralus"].unique()

array(['areas_y_corredores_de_alta_mixtura',
       'areas_y_corredores_de_media_mixtura', 'areas_de_baja_mixtura',
       'espacio_publico_proyectado', 'espacio_publico_existente', 'nan',
       'uso_dotacional'], dtype=object)

### Categories adjustment: Gravedad


In [19]:
joined_data["gravedad"].unique()

array(['con_heridos', 'solo_danos', 'con_muertos', 'herido', 'muerto'],
      dtype=object)

In [20]:
replacement_dict_gravedad = {"con_heridos": "herido", "con_muertos": "muerto"}

joined_data["gravedad"].replace(replacement_dict_gravedad, inplace=True)
joined_data["gravedad"].unique()

array(['herido', 'solo_danos', 'muerto'], dtype=object)

### Categories adjustment: Estrato


In [21]:
joined_data["estrato_pr"].unique()

array(['sin_estrato', 'tres', 'cuatro', 'seis', 'dos', 'uno', 'cinco',
       'nan'], dtype=object)

In [22]:
replacement_dict_estrato_pr = {
    "nan": 0,
    "sin_estrato": 0,
    "uno": 1,
    "dos": 2,
    "tres": 3,
    "cuatro": 4,
    "cinco": 5,
    "seis": 6,
}

joined_data["estrato_pr"].replace(replacement_dict_estrato_pr, inplace=True)

joined_data["estrato_pr"].unique()

array([0, 3, 4, 6, 2, 1, 5], dtype=int64)

### Categories adjustment: Tipo_geo_co


In [23]:
joined_data["tipo_geoco"].replace("malla_vial_p", "malla_vial", inplace=True)

In [24]:
# Apply the function to the specified column
joined_data["tipo_geoco"] = pd.DataFrame(joined_data["tipo_geoco"].apply(replace_malla))
joined_data["tipo_geoco"].unique()

array(['malla_vial', 'epm_sin_interior', 'malla_vial_cruce_invertido',
       'no_ubicada', 'epm_con_interior', 'malla_vial_aproximada', '0',
       'nomenclatura_con_interior', 'zona_rural'], dtype=object)

### Export file


In [25]:
joined_data.to_excel(config.CLEANED_DATA, index=False)